In [27]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()">
<input type="submit" value="Click here to view the raw code"></form>''')

# Shares Analysis UI
Select the stock you want to analyze

In [28]:
import api_requests as ar
import chart_studio.plotly as py
import plotly.graph_objs as go
import cufflinks as cf
import pandas as pd
import yfinance as yf
import holding_util as hu
import numpy as np

from ipywidgets import widgets, interact, interact_manual, HTML, HBox, VBox, Layout
from IPython.display import clear_output, display
from traitlets import traitlets

#File information
path=r"C:\Users\Marco\Documents"
file_name="investment_shares.csv"
df = ar.load_csv(path, file_name)
holdings_list = ar.create_shares_list(df)

In [29]:
def on_ticker_change(change):
    if ticker_dp.value:
        clear_output()
        display(ticker_dp)
        try:
            tick = yf.Ticker(ticker_dp.value)
            info = tick.info
            info_dict = {}
            value_list = ['shortName', 'sector']
            info_dict = {"Name": info.get('shortName'),
                         "Sector": info.get('sector'),
                         "Industry": info.get('industry'),
                         "Payout Ratio": info.get('payoutRatio'),
                         "Div Yield": info.get('dividendYield'),
                         "Market": info.get('exchange'),
                         "P/E": info.get('pegRatio'),
                         "Price": info.get('regularMarketPrice')}
            info_df = pd.DataFrame.from_dict(info_dict,
                                             orient='index',
                                             columns=["Details"])
            display(info_df)
            ticker_plot(tick)

        except (IndexError, AttributeError):
            print("No information available for {}".format(str(ticker_dp.value)))

def ticker_plot(tick, period="1y", interval="1d"):
    hist = tick.history(period=period, interval=interval)['Close']
    display(hist.iplot(yTitle="Price [{}]".format(tick.info['currency']),
                       kind="scatter"))

ticker_dp = widgets.Dropdown(options=holdings_list,
                             description="Holdings:",
                             disabled=False,
                             value=None)
ticker_dp.observe(on_ticker_change, names="value")
display(ticker_dp)

Dropdown(description='Holdings:', options=('ATVI', 'WM', 'NOBL', 'VOO', 'MU', 'BAM', 'QQQ', 'YUM', 'HPQ', 'LAZ…

,Details
Name,"Activision Blizzard, Inc"
Sector,Communication Services
Industry,Electronic Gaming & Multimedia
Payout Ratio,0.1897
Div Yield,0.007
Market,NMS
P/E,4.54
Price,56.29


## Portfolio Performance
Check out the performance of your stocks

In [30]:
#Creating Stocks dataframe
df = ar.compile_shares_df(df)
stock_df = ar.df_by_type(df, 'Stock')
weekend_check = ar.is_weekend()
history_df = ar.create_history_summary(stock_df.holding, period='1y', interval='1d')
last_close_value = ar.last_close(history_df)
combined_df = ar.join_shares_history_df(stock_df, history_df)
combined_df = ar.calc_share_gain(combined_df, weekend_check)
combined_df[['gain_USD','gain_perc']].head(20)

The last close date value is not for today's!


,gain_USD,gain_perc
ATVI,8.88,2.39
WM,43.23,13.08
MU,41.50,17.05
BAM,42.36,18.48
YUM,12.26,6.29
HPQ,-1.82,-1.13
LAZ,-25.58,-23.59
HUBS,80.68,76.66
INTC,31.12,31.90
AMD,280.70,304.25


In [31]:
# interact_manual=interact_manual.options(manual_name="Run")
# @interact_manual(Holding=df_btn.holdings_list)
# def ticker_info(Holding):
#     tick = yf.Ticker(Holding)
#     info = tick.info
#     info_dict = {"name": info['shortName'],
#                  "sector": info['sector'],
#                  "industry": info['industry'],
#                  "payout_perc": round(info['payoutRatio'] * 100, 2),
#                  "div_yield_perc": round(info['dividendYield'] * 100, 2),
#                  "xchange_mkt": info['exchange'],
#                  "pe_ratio": info['pegRatio'],
#                  "price_USD": info['regularMarketPrice']}
#     info_df = pd.DataFrame.from_dict(info_dict,
#                                      orient='index',
#                                      columns=[str(Holding)])
#     return info_df

In [32]:
# class LoadedButton(widgets.Button):
#     """A button that can holds a value as a attribute."""

#     def __init__(self, path=None, file_name=None, df=None, *args, **kwargs):
#         super(LoadedButton, self).__init__(*args, **kwargs)
#         # Create the value attribute.
#         self.path=path
#         self.file_name=file_name
#         self.add_traits(df=traitlets.Any(df))
        
# def return_holding_list(ex):
#     clear_output()
#     display(df_btn)
#     ex.df = ar.load_csv(ex.path, ex.file_name)
#     ex.holdings_list = ex.df['holding']

# df_btn = LoadedButton(description="Load CSV",
#                       path=path,
#                       file_name=file_name)
# df_btn.on_click(return_holding_list)
# display(df_btn)